In [36]:
import pandas as pd
import numpy as np
import requests
import time
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from selenium import webdriver
from bs4 import BeautifulSoup
import json

chromedriver = "/Users/edwardlee/Desktop/chromedriver"

In [37]:
browser = webdriver.Chrome(executable_path = chromedriver)

In [3]:
# Get all years for 1996-2016
browser.get('http://stats.nba.com/league/player/')
HTML = browser.page_source
year_data = Selector(text=HTML).xpath("//select[@name='Season']/option/text()").extract()
time.sleep(2)

# Get all years for gamelogs from 1947-2016
browser.get('http://stats.nba.com/league/player/#!/gamelogs/')
HTML_logs = browser.page_source
logs_year = Selector(text=HTML_logs).xpath("//select[@name='Season']/option/text()").extract()
time.sleep(2)

# Remove first array that says "All Season"
logs_year = logs_year[1:]

In [4]:
### MERGE ALL GAME LOGS FROM 1996-97 to 2015-16

all_game_logs = []

for year in year_data:
    csv_file = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/datasets/gamelogs/gamelogs_%s.csv' % year
    df = pd.read_csv(csv_file)
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df['SEASON'] = [year for i in range(df.shape[0])]
    all_game_logs.append(df)
    
gamelogs_df = pd.concat(all_game_logs)
save_csv = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/datasets/merged/all_gamelogs.csv'
gamelogs_df.to_csv(save_csv)

In [38]:
gamelogs_df = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/merged/all_gamelogs.csv')

In [39]:
# Removed unwanted columns and percentages unneeded for analysis
gamelogs_df.drop(['Unnamed: 0', 'SEASON_ID', 'TEAM_ABBREVIATION', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'VIDEO_AVAILABLE'], axis=1, inplace=True)

In [40]:
gamelogs_df.head(2)

,PLAYER_ID,PLAYER_NAME,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,SEASON
0,204067,Jarell Eddie,Washington Wizards,21501221,2016-04-13,WAS vs. ATL,W,21,2,5,...,4,5,1,1,0,0,1,7,-3,2015-16
1,202066,Garrett Temple,Washington Wizards,21501221,2016-04-13,WAS vs. ATL,W,16,4,7,...,1,1,1,1,1,6,0,12,-20,2015-16


In [41]:
gamelogs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481327 entries, 0 to 481326
Data columns (total 25 columns):
PLAYER_ID      481327 non-null int64
PLAYER_NAME    481327 non-null object
TEAM_NAME      481327 non-null object
GAME_ID        481327 non-null int64
GAME_DATE      481327 non-null object
MATCHUP        481327 non-null object
WL             481327 non-null object
MIN            481327 non-null int64
FGM            481327 non-null int64
FGA            481327 non-null int64
FG3M           481327 non-null int64
FG3A           481327 non-null int64
FTM            481327 non-null int64
FTA            481327 non-null int64
OREB           481327 non-null int64
DREB           481327 non-null int64
REB            481327 non-null int64
AST            481327 non-null int64
STL            481327 non-null int64
BLK            481327 non-null int64
TOV            481327 non-null int64
PF             481327 non-null int64
PTS            481327 non-null int64
PLUS_MINUS     481327 non-null in

In [42]:
# Add in new column with FP calculation
gamelogs_df['FP'] = (gamelogs_df['FGM'] * 1.0) - ((gamelogs_df['FGA']-gamelogs_df['FGM']) * (9./11.)) + (gamelogs_df['FTM'] * 1.0) - \
((gamelogs_df['FTA']-gamelogs_df['FTM']) * (3./1.)) + (gamelogs_df['FG3M'] * 0.5) + (gamelogs_df['PTS'] * 1.0) + (gamelogs_df['REB'] * 1.2) + \
(gamelogs_df['AST'] * 1.5) + (gamelogs_df['STL'] * 2.0) + (gamelogs_df['BLK'] * 2.0) - (gamelogs_df['TOV'] * 1.0)

In [43]:
gamelogs_df.sort_values(['FP'], ascending=False).head(5)

,PLAYER_ID,PLAYER_NAME,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,SEASON,FP
261603,977,Kobe Bryant,Los Angeles Lakers,20500591,2006-01-22,LAL vs. TOR,W,42,28,46,...,6,2,3,1,3,1,81,25,2005-06,124.972727
8366,203076,Anthony Davis,New Orleans Pelicans,21500823,2016-02-21,NOP @ DET,W,43,24,34,...,20,4,0,1,2,1,59,17,2015-16,111.818182
197747,2405,Amar'e Stoudemire,Phoenix Suns,20800051,2008-11-05,PHX @ IND,W,44,17,21,...,11,6,5,2,4,3,49,11,2008-09,109.927273
294032,1717,Dirk Nowitzki,Dallas Mavericks,20400222,2004-12-02,DAL vs. HOU,W,49,15,32,...,16,2,3,4,1,2,53,12,2004-05,108.290909
228812,977,Kobe Bryant,Los Angeles Lakers,20600977,2007-03-16,LAL vs. POR,W,50,23,39,...,7,3,3,0,2,3,65,4,2006-07,103.809091


In [44]:
gamelogs_df.to_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/merged/gamelogs_clean_withFP.csv')

In [45]:
gamelogs_target = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/merged/gamelogs_clean_withFP.csv')

In [46]:
# Groupby by season, player id, player name and fantasy points with agg function of mean, std and var
gamelogs_mean_fp = gamelogs_target.groupby(['SEASON','PLAYER_ID','PLAYER_NAME'])[['FP']].mean().reset_index().rename(columns={'FP':'fp_mean'})
gamelogs_std_fp = gamelogs_target.groupby(['SEASON','PLAYER_ID','PLAYER_NAME'])[['FP']].std().reset_index().rename(columns={'FP':'fp_std'})
gamelogs_var_fp = gamelogs_target.groupby(['SEASON','PLAYER_ID','PLAYER_NAME'])[['FP']].var().reset_index().rename(columns={'FP':'fp_var'})

In [47]:
# Groupby by season, team, fantasy points with agg function of mean, std, var
gamelogs_team_mean_fp = gamelogs_target.groupby(['SEASON', 'TEAM_NAME', 'GAME_DATE'])[['FP']].mean().reset_index().rename(columns={'FP':'fp_team_mean'})

In [48]:
# Name is only indicator, so don't need that for now, can remove
gamelogs_std_fp = gamelogs_std_fp[['PLAYER_ID','SEASON','fp_std']]
gamelogs_var_fp = gamelogs_var_fp[['PLAYER_ID','SEASON','fp_var']]

In [49]:
# Merge mean, std and var together
gamelogs_target_final = gamelogs_mean_fp.merge(gamelogs_std_fp, how='outer', on=['PLAYER_ID', 'SEASON']).merge(gamelogs_var_fp, how='outer', on=['PLAYER_ID', 'SEASON'])

In [50]:
gamelogs_target_final.head()

,SEASON,PLAYER_ID,PLAYER_NAME,fp_mean,fp_std,fp_var
0,1996-97,2,Byron Scott,12.213579,9.329127,87.032602
1,1996-97,3,Grant Long,10.892867,8.042089,64.675203
2,1996-97,7,Dan Schayes,7.250505,9.791436,95.872218
3,1996-97,9,Sedale Threatt,8.388745,7.748233,60.035115
4,1996-97,15,Eric Piatkowski,10.481818,8.831147,77.989155


In [51]:
gamelogs_team_mean_fp.head()

,SEASON,TEAM_NAME,GAME_DATE,fp_team_mean
0,1996-97,Atlanta Hawks,1996-11-01,18.419192
1,1996-97,Atlanta Hawks,1996-11-02,13.151818
2,1996-97,Atlanta Hawks,1996-11-04,17.039669
3,1996-97,Atlanta Hawks,1996-11-05,21.091736
4,1996-97,Atlanta Hawks,1996-11-07,19.642045


In [52]:
# Save dummy target variables together
gamelogs_target_final.to_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/completed/gamelogs_fp_targets.csv')

In [53]:
# Save team fp mean
gamelogs_team_mean_fp.to_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/completed/gamelogs_fp_team_mean.csv')